In [196]:
import torch
from collections import Counter
import random
import re

file_path = '/Users/jaafarbendriss/First-LLM/dataset/black_hole_wiki.txt'

# Open the file and read its content
with open(file_path, 'r') as file:
    file_content = file.read()

raw_text = file_content

def add_start_end_sentence(text):
    return "start-of-sentence " + text.replace("."," end-of-sentence start-of-sentence")


def remove_citations(text):
    pattern = r"\[\d+\]"
    cleaned_text = re.sub(pattern, "", text)    
    return cleaned_text

def remove_spaces(text):
    pattern = r"\s+"
    cleaned_text = re.sub(pattern, " ", text)    
    return cleaned_text

def preprocess(text: str):
    return remove_spaces(remove_citations(add_start_end_sentence(text).lower().replace(" s ","").replace("\\","").replace("'"," ").replace('"',' ').replace(","," ").replace("\ s"," ").replace("\n"," ")))

preprocessed_text = preprocess(raw_text)

Counter(preprocessed_text.split(" ")).most_common(20)

preprocessed_text_array = preprocessed_text.split(" ")


print(preprocessed_text_array[0:100])



['start-of-sentence', 'object', 'that', 'has', 'a', 'no-return', 'boundary', 'for', 'other', 'uses', 'see', 'black', 'hole', '(disambiguation)', 'end-of-sentence', 'start-of-sentence', 'direct', 'radio', 'image', 'of', 'a', 'supermassive', 'black', 'hole', 'at', 'the', 'core', 'of', 'messier', '87', 'animated', 'simulation', 'of', 'a', 'schwarzschild', 'black', 'hole', 'with', 'a', 'galaxy', 'passing', 'behind', 'end-of-sentence', 'start-of-sentence', 'around', 'the', 'time', 'of', 'alignment', 'extreme', 'gravitational', 'lensing', 'of', 'the', 'galaxy', 'is', 'observed', 'end-of-sentence', 'start-of-sentence', 'a', 'black', 'hole', 'is', 'a', 'region', 'of', 'spacetime', 'where', 'gravity', 'is', 'so', 'strong', 'that', 'nothing', 'including', 'light', 'and', 'other', 'electromagnetic', 'waves', 'has', 'enough', 'energy', 'to', 'escape', 'it', 'end-of-sentence', 'start-of-sentence', 'einstein', 's', 'theory', 'of', 'general', 'relativity', 'predicts', 'that', 'a', 'sufficiently', 'co

In [197]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer("basic_english")

In [198]:
tokens = tokenizer(preprocessed_text)

In [199]:
preprocessed_text = preprocessed_text.replace("end-of-sentence start-of-sentence", "end-of-sentence69696969start-of-sentence")

In [200]:
splited_data = preprocessed_text.split("69696969")

train_data =  [(i,sentence) for i, sentence in zip( range(0,len(splited_data)), splited_data  )]

In [201]:
train_data[0:5]

[(0,
  'start-of-sentence object that has a no-return boundary for other uses see black hole (disambiguation) end-of-sentence'),
 (1,
  'start-of-sentence direct radio image of a supermassive black hole at the core of messier 87 animated simulation of a schwarzschild black hole with a galaxy passing behind end-of-sentence'),
 (2,
  'start-of-sentence around the time of alignment extreme gravitational lensing of the galaxy is observed end-of-sentence'),
 (3,
  'start-of-sentence a black hole is a region of spacetime where gravity is so strong that nothing including light and other electromagnetic waves has enough energy to escape it end-of-sentence'),
 (4,
  'start-of-sentence einstein s theory of general relativity predicts that a sufficiently compact mass can deform spacetime to form a black hole end-of-sentence')]

In [202]:
train_iter = iter(train_data)


In [203]:
next(train_iter)

(0,
 'start-of-sentence object that has a no-return boundary for other uses see black hole (disambiguation) end-of-sentence')

In [204]:
next(train_iter)

(1,
 'start-of-sentence direct radio image of a supermassive black hole at the core of messier 87 animated simulation of a schwarzschild black hole with a galaxy passing behind end-of-sentence')

In [205]:

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

In [206]:
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [207]:
vocab

Vocab()

In [208]:
vocab(tokens[0:10])

[4, 37, 10, 66, 6, 0, 186, 16, 68, 0]

In [209]:
token_map = vocab.get_stoi()

In [210]:
token_map["black"]

5

In [211]:
token_map["hole"]

8

In [212]:
token_map["not"]

41

In [213]:
import random

def mask_token_in_sentence(sentence, word_to_token):
    """
    Masks a random token in a given sentence and returns the masked sentence and the masked token.
    
    Args:
    - sentence (str): The sentence from which to mask a token.
    - word_to_token (dict): A dictionary mapping words to their respective tokens.
    
    Returns:
    - Tuple of (masked_sentence, masked_token)
    """
    words = sentence.split()  # Split sentence into words
    mask_index = random.randint(0, len(words) - 1)  # Randomly select a word to mask
    masked_word = words[mask_index]  # Get the word to mask
    masked_token = word_to_token.get(masked_word, None)  # Get the token for the masked word, if available
    masked_token = masked_word
    
    if masked_token is None:
        return None, None  # or handle unknown words differently
    
    # Create the masked sentence by replacing the selected word with a special <mask> token or similar
    words[mask_index] = '<mask>'
    masked_sentence = ' '.join(words)
    
    return masked_sentence, masked_token

In [214]:
type(token_map)

dict

In [215]:
# Example usage:
train_sentences = ["This is an example sentence.", "Here is another one.", "Machine learning is fun."]
train_sentences = splited_data[0:10]

word_to_token = {"This": 1, "is": 2, "an": 3, "example": 4, "sentence": 5, "Here": 6, "another": 7, "one": 8, "Machine": 9, "learning": 10, "fun": 11}
word_to_token = token_map

masked_sentences_and_targets = [mask_token_in_sentence(sentence, word_to_token) for sentence in train_sentences]

# Filter out any None values if the word was not found in word_to_token
masked_sentences_and_targets = [item for item in masked_sentences_and_targets if item[0] is not None and item[1] is not None]

# Now, masked_sentences_and_targets contains tuples of (masked_sentence, masked_token)
# You can split this into training and validation sets as needed

In [216]:
masked_sentences_and_targets

[('<mask> object that has a no-return boundary for other uses see black hole (disambiguation) end-of-sentence',
  'start-of-sentence'),
 ('start-of-sentence direct radio image of a supermassive black hole at the core of messier 87 animated simulation of a schwarzschild <mask> hole with a galaxy passing behind end-of-sentence',
  'black'),
 ('start-of-sentence around the time of alignment extreme gravitational lensing of the <mask> is observed end-of-sentence',
  'galaxy'),
 ('start-of-sentence a black hole is a region of <mask> where gravity is so strong that nothing including light and other electromagnetic waves has enough energy to escape it end-of-sentence',
  'spacetime'),
 ('start-of-sentence einstein s theory of general relativity predicts that a sufficiently compact mass can deform spacetime <mask> form a black hole end-of-sentence',
  'to'),
 ('start-of-sentence the boundary of no escape is <mask> the event horizon end-of-sentence',
  'called'),
 ('start-of-sentence <mask> bla

In [221]:
def word_to_token_wrapper(word: str) -> int:
    if word not in word_to_token:
        return 0
    return word_to_token[word]

In [224]:
def sentence_to_token(sentence)->int:

    
    return [word_to_token_wrapper(word) for word in sentence.replace("<mask> ","").split()]

In [225]:
masked_sentences_and_targets_tokens = [(sentence_to_token(sentence),word_to_token[masked_word]) for sentence, masked_word in masked_sentences_and_targets]

In [227]:
masked_sentences_and_targets_tokens[0:5]

[([37, 10, 66, 6, 0, 186, 16, 68, 0, 182, 5, 8, 0, 3], 4),
 ([4,
   172,
   447,
   125,
   2,
   6,
   53,
   5,
   8,
   36,
   1,
   750,
   2,
   586,
   488,
   0,
   0,
   2,
   6,
   64,
   8,
   32,
   6,
   141,
   870,
   713,
   3],
  5),
 ([4, 114, 1, 67, 2, 1085, 780, 27, 213, 2, 1, 12, 241, 3], 141),
 ([4,
   6,
   5,
   8,
   12,
   6,
   160,
   2,
   184,
   61,
   12,
   130,
   149,
   10,
   598,
   813,
   43,
   11,
   68,
   333,
   223,
   66,
   268,
   106,
   7,
   137,
   19,
   3],
  93),
 ([4,
   105,
   40,
   59,
   2,
   48,
   58,
   438,
   10,
   6,
   1983,
   104,
   20,
   33,
   757,
   93,
   140,
   6,
   5,
   8,
   3],
  7)]

In [ ]:
[
 ([37, 10, 66, 8, 0, 3], 4),
 ([4, 114, 1, 67, 2, 1085, 1, 12, 241, 3], 141)
]

In [232]:
val_sentences

[([4,
   852,
   51,
   59,
   9,
   523,
   93,
   438,
   10,
   26,
   412,
   193,
   57,
   52,
   32,
   1,
   120,
   369,
   14,
   6,
   5,
   170,
   2,
   6,
   94,
   567,
   247,
   7,
   63,
   20,
   3],
  90),
 ([4,
   172,
   447,
   125,
   2,
   6,
   53,
   5,
   8,
   36,
   1,
   750,
   2,
   586,
   488,
   0,
   0,
   2,
   6,
   64,
   8,
   32,
   6,
   141,
   870,
   713,
   3],
  5)]

In [231]:
from sklearn.model_selection import train_test_split

# Assuming train_data is a list of sentences
train_sentences, val_sentences = train_test_split(masked_sentences_and_targets_tokens, test_size=0.2, random_state=42)

# Now, apply the masking and preparation logic to both train_sentences and val_sentences

# Simple Pytorch model

In [233]:
import torch
import torch.nn as nn
import torch.optim as optim

class SimpleLM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(SimpleLM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, tokens):
        embedded = self.embedding(tokens)  # [batch_size, seq_len, embedding_dim]
        # Aggregate embeddings, e.g., by averaging
        aggregated = embedded.mean(dim=1)  # [batch_size, embedding_dim]
        hidden = self.relu(self.linear1(aggregated))  # [batch_size, hidden_dim]
        output = self.linear2(hidden)  # [batch_size, output_dim]
        return output


In [273]:
data = masked_sentences_and_targets_tokens

In [274]:
[len(d) for d,_ in data] 

[14, 27, 14, 28, 21, 11, 29, 18, 31, 23]

In [275]:
from torch.utils.data import Dataset, DataLoader

class TokenDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        tokens, target = self.data[idx]
        return torch.tensor(tokens, dtype=torch.long), torch.tensor(target, dtype=torch.long)

# Assuming `data` is your list of mappings
dataset = TokenDataset(data)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)


In [276]:
len(word_to_token)

2145

In [277]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    tokens, targets = zip(*batch)
    # Pad the sequences to the maximum length in the batch
    tokens_padded = pad_sequence([torch.tensor(seq) for seq in tokens], batch_first=True, padding_value=0)
    targets = torch.tensor(targets, dtype=torch.long)
    return tokens_padded, targets


In [278]:
# Hyperparameters
vocab_size = 2145  # Adjust based on your dataset
embedding_dim = 50
hidden_dim = 100
output_dim = vocab_size  # Same as vocab size for prediction

model = SimpleLM(vocab_size, embedding_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [279]:
#dataloader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)


In [280]:
dataset = TokenDataset(data)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)


In [281]:
num_epochs = 100  # Or whatever suits your dataset

for epoch in range(num_epochs):
    total_loss = 0
    for tokens, target in dataloader:
        optimizer.zero_grad()
        output = model(tokens)
        loss = criterion(output, target.view(-1))
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {total_loss/len(dataloader)}')


/var/folders/l4/bv3z08h91lggkmhcp2n7yzh80000gn/T/ipykernel_59605/763346120.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokens_padded = pad_sequence([torch.tensor(seq) for seq in tokens], batch_first=True, padding_value=0)


Epoch 1, Loss: 7.690216700236003
Epoch 2, Loss: 7.612291018168132
Epoch 3, Loss: 7.557553609212239
Epoch 4, Loss: 7.496306737263997
Epoch 5, Loss: 7.440363725026448
Epoch 6, Loss: 7.3489430745442705
Epoch 7, Loss: 7.268342018127441
Epoch 8, Loss: 7.243323644002278
Epoch 9, Loss: 7.117378234863281
Epoch 10, Loss: 7.0059231122334795
Epoch 11, Loss: 6.845883369445801
Epoch 12, Loss: 6.726955572764079
Epoch 13, Loss: 6.531888961791992
Epoch 14, Loss: 6.190521240234375
Epoch 15, Loss: 6.126465638478597
Epoch 16, Loss: 5.853911399841309
Epoch 17, Loss: 5.596406777699788
Epoch 18, Loss: 5.668986479441325
Epoch 19, Loss: 4.926265875498454
Epoch 20, Loss: 4.792790571848552
Epoch 21, Loss: 4.143568197886149
Epoch 22, Loss: 3.7135039965311685
Epoch 23, Loss: 3.68514076868693
Epoch 24, Loss: 3.4160588582356772
Epoch 25, Loss: 3.0346622467041016
Epoch 26, Loss: 2.7496185302734375
Epoch 27, Loss: 2.7707616488138833
Epoch 28, Loss: 2.5804885228474936
Epoch 29, Loss: 2.4217917919158936
Epoch 30, Loss:

In [282]:
sequence = sentence_to_token("black hole is the biggest object in the")

In [283]:
# Example sequence
#sequence = [37, 10, 66, 8, 0, 3]  # assuming this sequence with the masked token
sequence_tensor = torch.tensor(sequence).unsqueeze(0)  # Add batch dimension


In [284]:
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient computation
    output = model(sequence_tensor)
    predicted_token_id = output.argmax(dim=1).item()  # Get the index of the max log-probability


In [285]:
# Assuming you have a token_to_word dictionary
token_to_word = {token: word for word, token in word_to_token.items()}
predicted_word = token_to_word[predicted_token_id]

In [286]:
predicted_word

'black'

In [287]:
def predict_missing_word(sentence="black hole is the biggest object in the"):
    sequence = sentence_to_token(sentence)
    # Example sequence
    #sequence = [37, 10, 66, 8, 0, 3]  # assuming this sequence with the masked token
    sequence_tensor = torch.tensor(sequence).unsqueeze(0)  # Add batch dimension
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Disable gradient computation
        output = model(sequence_tensor)
        predicted_token_id = output.argmax(dim=1).item()  # Get the index of the max log-probability
    # Assuming you have a token_to_word dictionary
    token_to_word = {token: word for word, token in word_to_token.items()}
    predicted_word = token_to_word[predicted_token_id]
    return predicted_word

In [288]:
predict_missing_word("start-of-sentence")

'a'

In [289]:
next_word = "start-of-sentence"
words = []
for _ in range(20):
    words.append(next_word)
    next_word = predict_missing_word(next_word)

" ".join(words)

'start-of-sentence a black a black a black a black a black a black a black a black a black a'

In [291]:
predict_missing_word("all black holes are")

'stellar'